In [34]:
"""
HomeGuard Security System Simulator
Author: April Atkinson
Description: A smart home monitoring system that processes sensor readings
             and triggers alerts for security, safety, and comfort issues.
"""

import random
from datetime import datetime

HOME_MODES = ["HOME", "AWAY", "SLEEP"]
ALERT_SEVERITIES = ["LOW", "MEDIUM", "HIGH", "CRITICAL"]

current_mode = "AWAY"


In [35]:
import random
from datetime import datetime


In [64]:
import time


In [36]:
def create_sensor(sensor_id, location, sensor_type, threshold=None):
    """
    Creates a sensor data structure.
    """
    sensor = {
        "id": sensor_id,
        "location": location,
        "type": sensor_type,
        "threshold": threshold,
        "current_value": None
    }
    return sensor


def create_alert(severity, message, sensor_id, timestamp):
    """
    Creates an alert data structure.
    """
    alert = {
        "severity": severity,
        "message": message,
        "sensor_id": sensor_id,
        "timestamp": timestamp
    }
    return alert


In [37]:
# Initialize sensors for the Peterson home
sensors = [
    create_sensor("MOTION_001", "Living Room", "motion"),
    create_sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
    create_sensor("DOOR_001", "Front Door", "door"),
    create_sensor("SMOKE_001", "Bedroom", "smoke")
]


In [38]:
print(f"Initialized {len(sensors)} sensors")
for sensor in sensors:
    print(f"  - {sensor['id']}: {sensor['location']} ({sensor['type']})")


Initialized 4 sensors
  - MOTION_001: Living Room (motion)
  - TEMP_001: Kitchen (temperature)
  - DOOR_001: Front Door (door)
  - SMOKE_001: Bedroom (smoke)


In [39]:
def is_abnormal_reading(sensor, reading_value):
    """
    Checks if a sensor reading is abnormal based on sensor type.
    """
    sensor_type = sensor["type"]

    if sensor_type == "temperature":
        # Too cold or too hot
        if reading_value < 35 or reading_value > 95:
            return True
        else:
            return False

    elif sensor_type == "motion":
        # Motion detected is abnormal
        return reading_value is True

    elif sensor_type == "door":
        # Door open is abnormal
        return reading_value == "OPEN"

    elif sensor_type == "smoke":
        # Smoke detected is abnormal
        return reading_value == "DETECTED"

    else:
        return False


In [40]:
def should_trigger_security_alert(sensor, reading_value, system_mode):
    """
    Determines if a security alert should be triggered.
    """
    sensor_type = sensor["type"]

    if system_mode == "AWAY":
        if sensor_type == "motion" and reading_value is True:
            return True
        if sensor_type == "door" and reading_value == "OPEN":
            return True

    return False


In [41]:
# Test temperature sensor
test_sensor = create_sensor("TEMP_TEST", "Test Room", "temperature", threshold=35)
print("34°F abnormal:", is_abnormal_reading(test_sensor, 34))  # True
print("68°F abnormal:", is_abnormal_reading(test_sensor, 68))  # False

# Test motion security logic
motion_sensor = create_sensor("MOTION_TEST", "Test Room", "motion")
print("Motion in AWAY triggers alert:",
      should_trigger_security_alert(motion_sensor, True, "AWAY"))  # True
print("Motion in HOME triggers alert:",
      should_trigger_security_alert(motion_sensor, True, "HOME"))  # False


34°F abnormal: True
68°F abnormal: False
Motion in AWAY triggers alert: True
Motion in HOME triggers alert: False


In [43]:
def generate_reading(sensor):
    """
    Generates a realistic reading for a sensor.
    """
    sensor_type = sensor["type"]

    if sensor_type == "temperature":
        # Temperature between 30°F and 100°F
        return random.randint(30, 100)

    elif sensor_type == "motion":
        # Motion detected or not
        return random.choice([True, False])

    elif sensor_type == "door":
        # Door state
        return random.choice(["OPEN", "CLOSED"])

    elif sensor_type == "smoke":
        # Smoke state
        return random.choice(["CLEAR", "DETECTED"])

    else:
        return None


In [44]:
# Test reading generation
test_sensor = sensors[0]  # First sensor (motion)
reading = generate_reading(test_sensor)
print(f"Generated reading for {test_sensor['location']}: {reading}")


Generated reading for Living Room: False


In [80]:
def process_reading(sensor, reading_value, system_mode):
    """
    Processes a sensor reading and determines if alerts are needed.
    Returns a list of alert dictionaries.
    """
    alerts = []
    timestamp = datetime.now().strftime("%H:%M:%S")

    sensor_type = sensor["type"]
    location = sensor["location"]

    # 1) SECURITY ALERTS (motion / door in AWAY mode)
    if should_trigger_security_alert(sensor, reading_value, system_mode):
        alert = create_alert(
            severity="HIGH",
            message=f"SECURITY: {location} {sensor_type} triggered while in {system_mode} mode!",
            sensor_id=sensor["id"],
            timestamp=timestamp
        )
        alerts.append(alert)

    # 2) SAFETY ALERTS (temperature extremes, smoke)
    if is_abnormal_reading(sensor, reading_value):
        if sensor_type == "temperature":
            alert = create_alert(
                severity="CRITICAL",
                message=f"SAFETY: Extreme temperature detected in {location}!",
                sensor_id=sensor["id"],
                timestamp=timestamp
            )
            alerts.append(alert)

        elif sensor_type == "smoke":
            alert = create_alert(
                severity="CRITICAL",
                message=f"SAFETY: Smoke detected in {location}!",
                sensor_id=sensor["id"],
                timestamp=timestamp
            )
            alerts.append(alert)

    # 3) COMFORT NOTIFICATIONS (only in HOME mode)
    if system_mode == "HOME" and sensor_type == "temperature":
        if reading_value < 65 or reading_value > 75:
            alert = create_alert(
                severity="LOW",
                message=f"COMFORT: Temperature in {location} is {reading_value}°F (outside 65–75°F range).",
                sensor_id=sensor["id"],
                timestamp=timestamp
            )
            alerts.append(alert)

    return alerts


In [67]:
# Test processing logic
motion_sensor = create_sensor("MOTION_TEST", "Test Room", "motion")
alerts = process_reading(motion_sensor, True, "AWAY")

print("Alerts triggered:", len(alerts))
if alerts:
    print(alerts[0])


Alerts triggered: 1
{'severity': 'HIGH', 'message': 'SECURITY: Test Room motion triggered while in AWAY mode!', 'sensor_id': 'MOTION_TEST', 'timestamp': '16:58:59'}


In [68]:
def trigger_alert(alert):
    """
    Displays an alert to the user.
    """
    severity_symbol = {
        "LOW": "ℹ️",
        "MEDIUM": "⚠️",
        "HIGH": "🚨",
        "CRITICAL": "🔥"
    }

    symbol = severity_symbol.get(alert["severity"], "⚠️")
    print(f"[ALERT!] {symbol} {alert['severity']}: {alert['message']}")


def log_event(message, timestamp=None):
    """
    Logs an event to the console.
    """
    if timestamp is None:
        timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[LOG] [{timestamp}] {message}")


In [69]:
# Create a fake alert and display it
test_alert = create_alert("HIGH", "SECURITY: Front Door opened while in AWAY mode!", "DOOR_001", "12:34:56")
trigger_alert(test_alert)

# Test logging
log_event("Sending notification to homeowner...", "12:34:56")


[ALERT!] 🚨 HIGH: SECURITY: Front Door opened while in AWAY mode!
[LOG] [12:34:56] Sending notification to homeowner...


In [70]:
class Sensor:
    """
    Represents a sensor in the HomeGuard system.
    """

    def __init__(self, sensor_id, location, sensor_type, threshold=None):
        self.id = sensor_id
        self.location = location
        self.type = sensor_type
        self.threshold = threshold
        self.current_value = None

    def read(self):
        sensor_dict = {
            "id": self.id,
            "location": self.location,
            "type": self.type,
            "threshold": self.threshold
        }
        self.current_value = generate_reading(sensor_dict)
        return self.current_value

    def isAbnormal(self):
        if self.current_value is None:
            return False

        sensor_dict = {
            "id": self.id,
            "location": self.location,
            "type": self.type,
            "threshold": self.threshold
        }
        return is_abnormal_reading(sensor_dict, self.current_value)

    def reset(self):
        self.current_value = None

    def __str__(self):
        status = "No reading" if self.current_value is None else str(self.current_value)
        return f"{self.id} ({self.location}): {status}"



In [71]:
test_sensor = Sensor("TEST_001", "Test Room", "temperature", threshold=35)
print(test_sensor.id)
print(test_sensor.location)
print(test_sensor.type)
print(test_sensor.threshold)
print(test_sensor.current_value)
print(test_sensor)


TEST_001
Test Room
temperature
35
None
TEST_001 (Test Room): No reading


In [72]:
    def read(self):
        """
        Generates and stores a new reading for this sensor.
        Returns the reading value.
        """
        # Convert this object into the dict format your generate_reading() expects
        sensor_dict = {
            "id": self.id,
            "location": self.location,
            "type": self.type,
            "threshold": self.threshold
        }

        self.current_value = generate_reading(sensor_dict)
        return self.current_value


In [73]:
test_sensor = Sensor("TEST_002", "Test Room", "door")
value = test_sensor.read()
print("Reading:", value)
print("Stored current_value:", test_sensor.current_value)
print("Sensor:", test_sensor)


Reading: CLOSED
Stored current_value: CLOSED
Sensor: TEST_002 (Test Room): CLOSED


In [74]:
    def isAbnormal(self):
        """
        Checks if the current reading is abnormal.
        """
        if self.current_value is None:
            return False

        sensor_dict = {
            "id": self.id,
            "location": self.location,
            "type": self.type,
            "threshold": self.threshold
        }

        return is_abnormal_reading(sensor_dict, self.current_value)


In [75]:
# Test abnormal detection
test_sensor = Sensor("TEST_003", "Test Room", "temperature", threshold=35)

test_sensor.current_value = 30
print("30°F abnormal:", test_sensor.isAbnormal())  # True

test_sensor.current_value = 70
print("70°F abnormal:", test_sensor.isAbnormal())  # False


30°F abnormal: True
70°F abnormal: False


In [81]:
def run_simulation(duration_minutes=3, system_mode="AWAY"):
    print("=" * 50)
    print("=== HomeGuard Security System ===")
    print("=" * 50)
    print(f"Mode: {system_mode}\n")

    sensors = [
        Sensor("MOTION_001", "Living Room", "motion"),
        Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
        Sensor("DOOR_001", "Front Door", "door"),
        Sensor("SMOKE_001", "Bedroom", "smoke")
    ]

    for _ in range(duration_minutes):
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"\nTime: {current_time}")

        for sensor in sensors:
            reading = sensor.read()

            # Display the reading
            if sensor.type == "temperature":
                status = "Normal" if 65 <= reading <= 75 else "Abnormal"
                print(f"[READING] {sensor.location} Temperature: {reading}°F ({status})")
            elif sensor.type == "motion":
                status = "DETECTED" if reading else "No activity"
                print(f"[READING] {sensor.location} Motion: {status}")
            elif sensor.type == "door":
                print(f"[READING] {sensor.location}: {reading}")
            elif sensor.type == "smoke":
                print(f"[READING] {sensor.location} Smoke: {reading}")

            # Convert sensor object -> dict for process_reading()
            sensor_dict = {
                "id": sensor.id,
                "location": sensor.location,
                "type": sensor.type,
                "threshold": sensor.threshold
            }

            alerts = process_reading(sensor_dict, reading, system_mode)

            for alert in alerts:
                trigger_alert(alert)
                if alert["severity"] in ["HIGH", "CRITICAL"]:
                    log_event("Sending notification to homeowner...")

        # optional: uncomment to slow down output
        # import time
        # time.sleep(1)

    print("\n" + "=" * 50)
    print("Simulation complete!")
    print("=" * 50)


In [84]:
run_simulation(duration_minutes=3, system_mode="AWAY")

=== HomeGuard Security System ===
Mode: AWAY


Time: 17:21:44
[READING] Living Room Motion: DETECTED
[ALERT!] 🚨 HIGH: SECURITY: Living Room motion triggered while in AWAY mode!
[LOG] [17:21:44] Sending notification to homeowner...
[READING] Kitchen Temperature: 43°F (Abnormal)
[READING] Front Door: CLOSED
[READING] Bedroom Smoke: CLEAR

Time: 17:21:44
[READING] Living Room Motion: DETECTED
[ALERT!] 🚨 HIGH: SECURITY: Living Room motion triggered while in AWAY mode!
[LOG] [17:21:44] Sending notification to homeowner...
[READING] Kitchen Temperature: 41°F (Abnormal)
[READING] Front Door: OPEN
[ALERT!] 🚨 HIGH: SECURITY: Front Door door triggered while in AWAY mode!
[LOG] [17:21:44] Sending notification to homeowner...
[READING] Bedroom Smoke: DETECTED
[ALERT!] 🔥 CRITICAL: SAFETY: Smoke detected in Bedroom!
[LOG] [17:21:44] Sending notification to homeowner...

Time: 17:21:44
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 94°F (Abnormal)
[READING] Front Door: OPEN


In [85]:
run_simulation(duration_minutes=3, system_mode="HOME")


=== HomeGuard Security System ===
Mode: HOME


Time: 17:21:49
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 40°F (Abnormal)
[ALERT!] ℹ️ LOW: COMFORT: Temperature in Kitchen is 40°F (outside 65–75°F range).
[READING] Front Door: OPEN
[READING] Bedroom Smoke: DETECTED
[ALERT!] 🔥 CRITICAL: SAFETY: Smoke detected in Bedroom!
[LOG] [17:21:49] Sending notification to homeowner...

Time: 17:21:49
[READING] Living Room Motion: DETECTED
[READING] Kitchen Temperature: 74°F (Normal)
[READING] Front Door: OPEN
[READING] Bedroom Smoke: DETECTED
[ALERT!] 🔥 CRITICAL: SAFETY: Smoke detected in Bedroom!
[LOG] [17:21:49] Sending notification to homeowner...

Time: 17:21:49
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 86°F (Abnormal)
[ALERT!] ℹ️ LOW: COMFORT: Temperature in Kitchen is 86°F (outside 65–75°F range).
[READING] Front Door: CLOSED
[READING] Bedroom Smoke: CLEAR

Simulation complete!


In [83]:
## Explanation

This program simulates a home security system by creating sensors for motion, temperature, doors, and smoke. It generates realistic sensor readings and uses if/else logic to determine abnormal conditions and trigger alerts based on the system mode (HOME or AWAY). When alerts occur, the system prints formatted alert messages and logs notification events.


SyntaxError: invalid syntax (813185034.py, line 3)